<h2>This notebook will be mainly used for the capstone project.<h2>
<p>I am happy to share my project in data science professional certificate IBM<p> 

In [1]:
import pandas as pd 
import numpy as np
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import folium
import requests 
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans

#  Building dataframe for Toronto 

In [2]:
df_PostalCode = pd.read_csv('Postal_Code.csv')
df_PostalCode.head()

,Postal Code,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park,Harbourfront"
3,M6A,North York,"Lawrence Manor,Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government"


In [3]:
df_PostalCode.shape

(103, 3)

In [4]:
df_geo = pd.read_csv('Geospatial_Coordinates.csv')
df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [5]:
neighborhoods = df_PostalCode.groupby(by = 'Postal Code')
neighborhoods = df_PostalCode.merge(df_geo, how ='left')
neighborhoods

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park,Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park,Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway,Montgomery Road,Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,Business reply mail Processing Centre 969 East...,43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South,King's Mill Park ,Sunnylea,Humb...",43.636258,-79.498509


In [6]:
print('The dataframe has {} boroughs and {} neighborhoods.'.format(
        len(neighborhoods['Borough'].unique()),
        neighborhoods.shape[0]
    )
)

The dataframe has 11 boroughs and 103 neighborhoods.


# Visualizing neighborhood in Toronto

In [7]:
address = 'Toronto, Ontario'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [8]:
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lng, borough, neighborhood in zip(neighborhoods['Latitude'], neighborhoods['Longitude'], neighborhoods['Borough'], neighborhoods['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='#32a89d',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto


# Explore neighborhoods in Norht York

In [9]:
NY_df = neighborhoods[neighborhoods['Borough'] == 'North York'].reset_index(drop=True)
NY_df.head()

,Postal Code,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763
3,M3B,North York,"Don Mills,North",43.745906,-79.352188
4,M6B,North York,Glencairn,43.709577,-79.445073


In [10]:
NY_df.shape

(24, 5)

In [11]:
address = 'North York,Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of North York Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of North York Toronto are 43.7543263, -79.44911696639593.


# Visualizing neighborhood in North York

In [12]:
# create map of North York using latitude and longitude values
map_NY = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(NY_df['Latitude'], NY_df['Longitude'], NY_df['Neighborhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_NY)  
    
map_NY

In [13]:
CLIENT_ID = 'MBBKD0HFFWF5PCLNLKDDZ03TXY23UBCJWUUIGHIXG2VLSMDZ' # your Foursquare ID
CLIENT_SECRET = 'JZWLUUEQYWUQZARLKV2WDPFN0J2IW3GAYQJOQTBDN2CHVQH1' # your Foursquare Secret
VERSION = '20203030' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: MBBKD0HFFWF5PCLNLKDDZ03TXY23UBCJWUUIGHIXG2VLSMDZ
CLIENT_SECRET:JZWLUUEQYWUQZARLKV2WDPFN0J2IW3GAYQJOQTBDN2CHVQH1


# The top 100 venues that are in Norht York within a radius of 500 meters.

In [14]:
LIMIT = 100
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

# Neighborhood in Norht York within a radius of 500 meters.

In [15]:
NY_venues = getNearbyVenues(names=NY_df['Neighborhood'],
                                   latitudes=NY_df['Latitude'],
                                   longitudes=NY_df['Longitude']
                                  )

Parkwoods
Victoria Village
Lawrence Manor,Lawrence Heights
Don Mills,North
Glencairn
Don Mills South,Flemingdon Park
Hillcrest Village
Bathurst Manor,Wilson Heights,Downsview North
Fairview,Henry Farm,Oriole
Northwood Park,York University
Bayview Village
Downsview,CFB Toronto
York Mills,Silver Hills
Downsview West
North Park,Maple Leaf Park,Upwood Park
Humber Summit
Willowdale,Newtonbrook
Downsview,Central
Bedford Park,Lawrence Manor East
Humberlea,Emery
Willowdale South
Downsview Northwest
York Mills West
Willowdale West


# Venues that are in Norht York within a radius of 500 meters.

In [16]:
print(NY_venues.shape)
NY_venues.head()

(239, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


In [17]:
print('There are {} uniques categories.'.format(len(NY_venues['Venue Category'].unique())))

There are 104 uniques categories.


In [18]:
# one hot encoding
NY_onehot = pd.get_dummies(NY_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
NY_onehot['Neighborhood'] = NY_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [NY_onehot.columns[-1]] + list(NY_onehot.columns[:-1])
NY_onehot = NY_onehot[fixed_columns]

NY_onehot.head()

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Parkwoods,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [19]:
NY_onehot.shape

(239, 105)

In [20]:
NY_grouped = NY_onehot.groupby('Neighborhood').mean().reset_index()
NY_grouped

,Neighborhood,Accessories Store,Airport,American Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Bakery,Bank,Bar,...,Supermarket,Supplement Shop,Sushi Restaurant,Tea Room,Thai Restaurant,Theater,Toy / Game Store,Video Game Store,Vietnamese Restaurant,Women's Store
0,"Bathurst Manor,Wilson Heights,Downsview North",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.000000,...,0.05,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Bayview Village,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.250000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,"Bedford Park,Lawrence Manor East",0.000000,0.0,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.041667,0.000000,0.041667,0.000000,0.000000,0.000000,0.000000,0.000000
3,"Don Mills South,Flemingdon Park",0.000000,0.0,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.000000,...,0.05,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,"Don Mills,North",0.000000,0.0,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Downsview Northwest,0.000000,0.0,0.000000,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Downsview West,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,"Downsview,CFB Toronto",0.000000,0.5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,"Downsview,Central",0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,"Fairview,Henry Farm,Oriole",0.000000,0.0,0.015385,0.000000,0.015385,0.000000,0.030769,0.030769,0.015385,...,0.00,0.015385,0.000000,0.030769,0.015385,0.015385,0.015385,0.015385,0.000000,0.015385


In [21]:
NY_grouped.shape

(23, 105)

# Display each neighborhood with the top 5 most common venues

In [22]:
num_top_venues = 5

for hood in NY_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = NY_grouped[NY_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Bathurst Manor,Wilson Heights,Downsview North----
                 venue  freq
0          Coffee Shop  0.10
1                 Bank  0.10
2             Pharmacy  0.05
3  Fried Chicken Joint  0.05
4                Diner  0.05


----Bayview Village----
                 venue  freq
0   Chinese Restaurant  0.25
1                 Bank  0.25
2                 Café  0.25
3  Japanese Restaurant  0.25
4    Accessories Store  0.00


----Bedford Park,Lawrence Manor East----
                venue  freq
0      Sandwich Place  0.08
1  Italian Restaurant  0.08
2         Coffee Shop  0.08
3          Restaurant  0.08
4    Greek Restaurant  0.04


----Don Mills South,Flemingdon Park----
          venue  freq
0           Gym  0.10
1   Coffee Shop  0.10
2    Restaurant  0.10
3    Beer Store  0.10
4  Concert Hall  0.05


----Don Mills,North----
                  venue  freq
0  Gym / Fitness Center  0.17
1    Athletics & Sports  0.17
2  Caribbean Restaurant  0.17
3        Baseball Field  0.17
4          

In [23]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

 ### The top 10 venues for each neighborhood.

In [24]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = NY_grouped['Neighborhood']

for ind in np.arange(NY_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(NY_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Bathurst Manor,Wilson Heights,Downsview North",Coffee Shop,Bank,Middle Eastern Restaurant,Shopping Mall,Pizza Place,Ice Cream Shop,Bridal Shop,Deli / Bodega,Diner,Restaurant
1,Bayview Village,Japanese Restaurant,Chinese Restaurant,Café,Bank,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
2,"Bedford Park,Lawrence Manor East",Coffee Shop,Sandwich Place,Italian Restaurant,Restaurant,Comfort Food Restaurant,Juice Bar,Café,Butcher,Indian Restaurant,Pet Store
3,"Don Mills South,Flemingdon Park",Restaurant,Gym,Beer Store,Coffee Shop,Discount Store,Italian Restaurant,Japanese Restaurant,Asian Restaurant,Bike Shop,Concert Hall
4,"Don Mills,North",Japanese Restaurant,Gym / Fitness Center,Athletics & Sports,Caribbean Restaurant,Café,Baseball Field,Women's Store,Convenience Store,Cosmetics Shop,Deli / Bodega




# Cluster Neighborhoods

## Run *k*-means to cluster the neighborhood into 5 clusters.

In [25]:
kclusters = 5

NY_grouped_clustering = NY_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(NY_grouped_clustering)

kmeans.labels_[0:10] 

array([1, 1, 1, 1, 1, 1, 1, 3, 1, 1], dtype=int32)

In [26]:
neighborhoods_venues_sorted.insert(0, 'Cluster_Labels', kmeans.labels_)

NY_merged =NY_df

NY_merged = NY_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

NY_merged.head() 

,Postal Code,Borough,Neighborhood,Latitude,Longitude,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M3A,North York,Parkwoods,43.753259,-79.329656,3.0,Park,Food & Drink Shop,Convenience Store,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
1,M4A,North York,Victoria Village,43.725882,-79.315572,1.0,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,M6A,North York,"Lawrence Manor,Lawrence Heights",43.718518,-79.464763,1.0,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Vietnamese Restaurant,Accessories Store,Supermarket
3,M3B,North York,"Don Mills,North",43.745906,-79.352188,1.0,Japanese Restaurant,Gym / Fitness Center,Athletics & Sports,Caribbean Restaurant,Café,Baseball Field,Women's Store,Convenience Store,Cosmetics Shop,Deli / Bodega
4,M6B,North York,Glencairn,43.709577,-79.445073,1.0,Park,Pizza Place,Pub,Japanese Restaurant,Women's Store,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store


In [27]:
NY_merged = NY_merged.fillna(0)
NY_merged.Cluster_Labels.astype(int)
NY_merged.dtypes

Postal Code                object
Borough                    object
Neighborhood               object
Latitude                  float64
Longitude                 float64
Cluster_Labels            float64
1st Most Common Venue      object
2nd Most Common Venue      object
3rd Most Common Venue      object
4th Most Common Venue      object
5th Most Common Venue      object
6th Most Common Venue      object
7th Most Common Venue      object
8th Most Common Venue      object
9th Most Common Venue      object
10th Most Common Venue     object
dtype: object

# Visualizing the resulting clusters

In [28]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

In [29]:
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]
#colors_array = colors_array.round(2)*10
#colors_array = colors_array.astype(int)

print(colors_array)


[[5.00000000e-01 0.00000000e+00 1.00000000e+00 1.00000000e+00]
 [1.96078431e-03 7.09281308e-01 9.23289106e-01 1.00000000e+00]
 [5.03921569e-01 9.99981027e-01 7.04925547e-01 1.00000000e+00]
 [1.00000000e+00 7.00543038e-01 3.78411050e-01 1.00000000e+00]
 [1.00000000e+00 1.22464680e-16 6.12323400e-17 1.00000000e+00]]


In [30]:
# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(NY_merged['Latitude'], NY_merged['Longitude'], NY_merged['Neighborhood'], NY_merged['Cluster_Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color = rainbow[int(cluster)-1],
       # color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[int(cluster)-1],
        #fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Cluster 1

In [31]:
NY_merged.loc[NY_merged['Cluster_Labels'] == 0, 
                          NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
12,North York,0.0,Cafeteria,Martial Arts Dojo,Women's Store,Coffee Shop,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store
16,North York,0.0,0,0,0,0,0,0,0,0,0,0


## Cluster 2

In [32]:
NY_merged.loc[NY_merged['Cluster_Labels'] == 1, 
                     NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,North York,1.0,Coffee Shop,Hockey Arena,Portuguese Restaurant,Intersection,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
2,North York,1.0,Clothing Store,Furniture / Home Store,Women's Store,Miscellaneous Shop,Boutique,Coffee Shop,Event Space,Vietnamese Restaurant,Accessories Store,Supermarket
3,North York,1.0,Japanese Restaurant,Gym / Fitness Center,Athletics & Sports,Caribbean Restaurant,Café,Baseball Field,Women's Store,Convenience Store,Cosmetics Shop,Deli / Bodega
4,North York,1.0,Park,Pizza Place,Pub,Japanese Restaurant,Women's Store,Clothing Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
5,North York,1.0,Restaurant,Gym,Beer Store,Coffee Shop,Discount Store,Italian Restaurant,Japanese Restaurant,Asian Restaurant,Bike Shop,Concert Hall
6,North York,1.0,Golf Course,Mediterranean Restaurant,Athletics & Sports,Pool,Dog Run,Diner,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop
7,North York,1.0,Coffee Shop,Bank,Middle Eastern Restaurant,Shopping Mall,Pizza Place,Ice Cream Shop,Bridal Shop,Deli / Bodega,Diner,Restaurant
8,North York,1.0,Clothing Store,Coffee Shop,Fast Food Restaurant,Japanese Restaurant,Food Court,Bank,Bakery,Juice Bar,Tea Room,Deli / Bodega
9,North York,1.0,Miscellaneous Shop,Massage Studio,Caribbean Restaurant,Bar,Coffee Shop,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store
10,North York,1.0,Japanese Restaurant,Chinese Restaurant,Café,Bank,Women's Store,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store


## Cluster 3

In [33]:
NY_merged.loc[NY_merged['Cluster_Labels'] == 2, 
                     NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
19,North York,2.0,Baseball Field,Women's Store,Distribution Center,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega,Department Store,Dessert Shop


## Cluster 4

In [34]:
NY_merged.loc[NY_merged['Cluster_Labels'] == 3, 
                     NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,North York,3.0,Park,Food & Drink Shop,Convenience Store,Diner,Coffee Shop,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Cosmetics Shop,Deli / Bodega
11,North York,3.0,Airport,Park,Women's Store,Discount Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
22,North York,3.0,Park,Convenience Store,Bank,Women's Store,Discount Store,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Cosmetics Shop,Deli / Bodega


## Cluster 5

In [35]:
NY_merged.loc[NY_merged['Cluster_Labels'] == 4, 
                     NY_merged.columns[[1] + list(range(5, NY_merged.shape[1]))]]

,Borough,Cluster_Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,North York,4.0,Pizza Place,Empanada Restaurant,Women's Store,Diner,Comfort Food Restaurant,Concert Hall,Construction & Landscaping,Convenience Store,Cosmetics Shop,Deli / Bodega
